In [ ]:
from src.motion_control.MotionControl import MotionControl
from src.motion_control.ThymioController import ThymioController
#Vision 

#Global navigation 
path = [(19, 1), (20, 1), (21, 1), (21, 2), (21, 3), (21, 4)]


#Motion control, filtering, local navigation
thymio = ThymioController()    
    
    



In [ ]:
try:
    # Define path (list of waypoints)
    
    thymio.connect(timeout=5)
    motion_control = MotionControl(thymio, path)
    motion_control.execute_path()

finally:
    thymio.disconnect()